# FAQ Update Database Setup - Single Database Architecture

## Purpose

This notebook creates the **faq_update.db** database with all tables in a single unified schema.

### Schema v4.0 - Single Database Architecture

**Major Improvement**: All tables in ONE database = REAL foreign keys enforced by SQLite!

#### Tables (7 total):
1. **content_repo** - Source content (documents, pages, markdown)
2. **faq_questions** - FAQ questions master table
3. **faq_answers** - FAQ answers master table
4. **content_change_log** - Content version history with FK to content_repo
5. **faq_content_map** - FAQ-content mapping with FK to all relevant tables
6. **content_diffs** - Diff storage with FK to content_change_log
7. **schema_version** - Version tracking metadata

#### Views (8 total):
- v_document_structure_changes
- v_current_valid_faqs
- v_faq_validity_timeline
- v_faqs_needing_review
- v_content_change_summary
- v_latest_content_state
- v_content_changes_with_diffs
- v_pending_diffs
- v_diff_processing_stats

#### Trigger (1 total):
- trg_content_diffs_only_for_edits

### Benefits of Single Database:
✅ Real foreign key enforcement (CASCADE, SET NULL)
✅ ACID transactions across all tables
✅ Simpler queries (no ATTACH DATABASE)
✅ Better performance (single connection)
✅ Easier backup (one file)

## When to Run
- Initial setup
- Database reset
- Schema upgrade from v3.x to v4.0
- Testing with sample data

## Step 1: Configuration and Imports

In [ ]:
import sqlite3
import os
from pathlib import Path
import gzip
import json
from datetime import datetime

# Configuration
DATABASE_NAME = 'faq_update.db'
DATABASE_DIR = Path('../databases')
SQL_DIR = Path('../sql')
SCHEMA_FILE = SQL_DIR / 'create_schema_v4.sql'

# Full database path
DB_PATH = DATABASE_DIR / DATABASE_NAME

# Create directories if they don't exist
DATABASE_DIR.mkdir(exist_ok=True, parents=True)

print(f"Database will be created at: {DB_PATH.absolute()}")
print(f"Schema file: {SCHEMA_FILE.absolute()}")
print(f"Schema file exists: {SCHEMA_FILE.exists()}")

## Step 2: Remove Existing Database (Clean Start)

In [ ]:
# Remove existing database for clean start
if DB_PATH.exists():
    print(f"Removing existing database: {DB_PATH}")
    os.remove(DB_PATH)
    print("✓ Existing database removed")
else:
    print("No existing database to remove")

## Step 3: Verify JSON1 Extension

In [ ]:
# Verify JSON1 extension is available
with sqlite3.connect(DB_PATH) as conn:
    try:
        result = conn.execute("SELECT json('{\"test\":true}')").fetchone()
        print("✓ JSON1 extension is available")
        print(f"  Test result: {result[0]}")
    except sqlite3.OperationalError as e:
        print("✗ JSON1 extension NOT available")
        print(f"  Error: {e}")
        print("  Schema v4.0 requires JSON1 for semantic_summary field")

## Step 4: Create Schema from SQL File

In [ ]:
# Read and execute schema SQL
print(f"Reading schema from: {SCHEMA_FILE}")
with open(SCHEMA_FILE, 'r', encoding='utf-8') as f:
    schema_sql = f.read()

print(f"Schema SQL size: {len(schema_sql)} characters")

# Execute schema creation
with sqlite3.connect(DB_PATH) as conn:
    print("\nExecuting schema creation...")
    conn.executescript(schema_sql)
    print("✓ Schema created successfully")
    
    # Verify foreign keys are enabled
    fk_status = conn.execute("PRAGMA foreign_keys").fetchone()[0]
    print(f"\nForeign keys enabled: {bool(fk_status)} (CRITICAL for v4.0)")
    if not fk_status:
        print("WARNING: Foreign keys are NOT enabled!")
        conn.execute("PRAGMA foreign_keys = ON")
        print("✓ Foreign keys enabled")

## Step 5: Verify Schema Components

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    # Count tables
    tables = conn.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='table' AND name NOT LIKE 'sqlite_%'
        ORDER BY name
    """).fetchall()
    
    print(f"Tables created: {len(tables)}")
    for table in tables:
        print(f"  - {table[0]}")
    
    # Count views
    views = conn.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='view'
        ORDER BY name
    """).fetchall()
    
    print(f"\nViews created: {len(views)}")
    for view in views:
        print(f"  - {view[0]}")
    
    # Count triggers
    triggers = conn.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='trigger'
        ORDER BY name
    """).fetchall()
    
    print(f"\nTriggers created: {len(triggers)}")
    for trigger in triggers:
        print(f"  - {trigger[0]}")
    
    # Count indexes
    indexes = conn.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='index' AND name NOT LIKE 'sqlite_%'
        ORDER BY name
    """).fetchall()
    
    print(f"\nIndexes created: {len(indexes)}")
    print("(Not listing all indexes - see create_schema_v4.sql for details)")
    
    # Check schema version
    version = conn.execute("""
        SELECT schema_version, description FROM schema_version
    """).fetchone()
    
    print(f"\nSchema Version: {version[0]}")
    print(f"Description: {version[1]}")

## Step 6: Insert Sample Data - content_repo

Simulating 3 pages from a PDF document

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Insert 3 pages from test document
    content_data = [
        (
            'Employee_Leave_Policy.pdf',
            1,
            '/path/to/markdown/page_1.md',
            '2025-01-01T00:00:00Z',
            '2025-01-01T00:00:00Z',
            15000,
            'pdf',
            'completed'
        ),
        (
            'Employee_Leave_Policy.pdf',
            2,
            '/path/to/markdown/page_2.md',
            '2025-01-01T00:00:00Z',
            '2025-01-01T00:00:00Z',
            16000,
            'pdf',
            'completed'
        ),
        (
            'Employee_Leave_Policy.pdf',
            3,
            '/path/to/markdown/page_3.md',
            '2025-01-01T00:00:00Z',
            '2025-01-01T00:00:00Z',
            14500,
            'pdf',
            'completed'
        ),
    ]
    
    conn.executemany("""
        INSERT INTO content_repo (
            raw_file_nme, raw_file_page_nbr, extracted_markdown_file_path,
            last_modified_dt, created_dt, file_size_bytes, content_type, processing_status
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, content_data)
    
    conn.commit()
    
    # Verify
    count = conn.execute("SELECT COUNT(*) FROM content_repo").fetchone()[0]
    print(f"✓ Inserted {count} records into content_repo")
    
    # Show the data
    print("\nSample data:")
    rows = conn.execute("""
        SELECT ud_source_file_id, raw_file_nme, raw_file_page_nbr, processing_status
        FROM content_repo
    """).fetchall()
    
    for row in rows:
        print(f"  ID: {row[0]}, File: {row[1]}, Page: {row[2]}, Status: {row[3]}")

## Step 7: Insert Sample Data - faq_questions

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Insert 10 sample questions
    questions = [
        ("What is the annual leave policy?", "Leave", 1),
        ("How do I request time off?", "Leave", 1),
        ("What are the sick leave benefits?", "Leave", 2),
        ("Can I carry over unused vacation days?", "Leave", 1),
        ("What is the maternity leave policy?", "Leave", 3),
        ("How does the flexible work arrangement work?", "WorkLife", 1),
        ("What are the company holidays?", "Leave", 1),
        ("How do I apply for unpaid leave?", "Leave", 2),
        ("What is the bereavement leave policy?", "Leave", 3),
        ("Can I take leave during probation period?", "Leave", 2),
    ]
    
    conn.executemany("""
        INSERT INTO faq_questions (question_txt, category, priority)
        VALUES (?, ?, ?)
    """, questions)
    
    conn.commit()
    
    # Verify
    count = conn.execute("SELECT COUNT(*) FROM faq_questions").fetchone()[0]
    print(f"✓ Inserted {count} records into faq_questions")
    
    # Show sample
    print("\nFirst 5 questions:")
    rows = conn.execute("""
        SELECT question_id, question_txt, category
        FROM faq_questions
        LIMIT 5
    """).fetchall()
    
    for row in rows:
        print(f"  Q{row[0]}: {row[1][:50]}... (Cat: {row[2]})")

## Step 8: Insert Sample Data - faq_answers

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Insert 10 sample answers (matching questions)
    answers = [
        ("Employees are entitled to 20 days of annual leave per year.", "text", 95),
        ("You can request time off through the HR portal or by submitting a leave form.", "text", 105),
        ("Sick leave is provided up to 10 days per year with medical certificate.", "text", 98),
        ("Yes, up to 5 unused vacation days can be carried over to the next year.", "text", 85),
        ("Maternity leave is 16 weeks, with the option to extend unpaid.", "text", 78),
        ("Flexible arrangements include remote work 2 days per week and flexible hours.", "text", 92),
        ("The company observes 11 public holidays plus additional shutdown days.", "text", 67),
        ("Unpaid leave requests must be submitted 30 days in advance through HR.", "text", 88),
        ("Bereavement leave is 3-5 days depending on relationship to the deceased.", "text", 82),
        ("Leave during probation is at manager's discretion, usually after 3 months.", "text", 91),
    ]
    
    conn.executemany("""
        INSERT INTO faq_answers (faq_answer_txt, answer_format, word_count)
        VALUES (?, ?, ?)
    """, answers)
    
    conn.commit()
    
    # Verify
    count = conn.execute("SELECT COUNT(*) FROM faq_answers").fetchone()[0]
    print(f"✓ Inserted {count} records into faq_answers")
    
    # Show sample
    print("\nFirst 5 answers:")
    rows = conn.execute("""
        SELECT answer_id, faq_answer_txt, word_count
        FROM faq_answers
        LIMIT 5
    """).fetchall()
    
    for row in rows:
        print(f"  A{row[0]}: {row[1][:50]}... ({row[2]} words)")

## Step 9: Insert Sample Data - content_change_log

Simulating content changes with different types

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Sample checksums (SHA-256 format - 64 hex characters)
    checksum_1 = 'a' * 64
    checksum_2 = 'b' * 64
    checksum_3 = 'c' * 64
    checksum_2_edited = 'd' * 64  # Page 2 edited
    
    # Insert initial creation records
    changes = [
        # Initial creation of all 3 pages
        (
            1, 'Employee_Leave_Policy.pdf', 1, checksum_1,
            'initial_creation', None, None, None, 'initial_load',
            '2025-01-01T00:00:00Z', '2025-01-01T00:00:00Z'
        ),
        (
            2, 'Employee_Leave_Policy.pdf', 2, checksum_2,
            'initial_creation', None, None, None, 'initial_load',
            '2025-01-01T00:00:00Z', '2025-01-01T00:00:00Z'
        ),
        (
            3, 'Employee_Leave_Policy.pdf', 3, checksum_3,
            'initial_creation', None, None, None, 'initial_load',
            '2025-01-01T00:00:00Z', '2025-01-01T00:00:00Z'
        ),
        # Page 2 content edited
        (
            2, 'Employee_Leave_Policy.pdf', 2, checksum_2_edited,
            'content_edit', 2, 2, checksum_2, 'content_update',
            '2025-01-15T10:30:00Z', '2025-01-15T10:30:00Z'
        ),
    ]
    
    conn.executemany("""
        INSERT INTO content_change_log (
            content_id, file_name, page_number, content_checksum,
            change_type, previous_content_id, previous_page_number, previous_checksum,
            change_trigger, detected_at, effective_date
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, changes)
    
    conn.commit()
    
    # Verify
    count = conn.execute("SELECT COUNT(*) FROM content_change_log").fetchone()[0]
    print(f"✓ Inserted {count} records into content_change_log")
    
    # Show sample
    print("\nChange log:")
    rows = conn.execute("""
        SELECT change_id, file_name, page_number, change_type, effective_date
        FROM content_change_log
        ORDER BY change_id
    """).fetchall()
    
    for row in rows:
        print(f"  Change {row[0]}: {row[1]} page {row[2]} - {row[3]} at {row[4]}")

## Step 10: Insert Sample Data - faq_content_map

Linking FAQs to content with validity tracking

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Get checksums for mapping
    checksum_1 = 'a' * 64
    checksum_2 = 'b' * 64  # Original page 2
    checksum_2_edited = 'd' * 64  # Edited page 2
    checksum_3 = 'c' * 64
    
    # Map first 5 FAQs to content
    mappings = [
        # Q1-Q2 on page 1 (still valid)
        (1, 1, checksum_1, 1, 1, 'Employee_Leave_Policy.pdf', 1, 1, 1, 0.95, 'automated'),
        (2, 2, checksum_1, 1, 1, 'Employee_Leave_Policy.pdf', 1, 1, 1, 0.92, 'automated'),
        
        # Q3-Q4 on page 2 - INVALIDATED by edit (change_id = 4)
        (3, 3, checksum_2, 2, 2, 'Employee_Leave_Policy.pdf', 2, 2, 0, 0.88, 'automated'),
        (4, 4, checksum_2, 2, 2, 'Employee_Leave_Policy.pdf', 2, 2, 0, 0.91, 'automated'),
        
        # Q5 on page 3 (still valid)
        (5, 5, checksum_3, 3, 3, 'Employee_Leave_Policy.pdf', 3, 3, 1, 0.89, 'automated'),
    ]
    
    # Insert mappings
    for mapping in mappings:
        if mapping[8] == 0:  # is_valid = 0
            # Invalidated FAQs
            conn.execute("""
                INSERT INTO faq_content_map (
                    question_id, answer_id, content_checksum,
                    current_content_id, current_page_number, current_file_name,
                    original_content_id, original_page_number,
                    is_valid, valid_until, invalidation_reason, invalidated_by_change_id,
                    confidence_score, generation_method
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, '2025-01-15T10:30:00Z', 'content_edited', 4, ?, ?)
            """, mapping)
        else:
            # Valid FAQs
            conn.execute("""
                INSERT INTO faq_content_map (
                    question_id, answer_id, content_checksum,
                    current_content_id, current_page_number, current_file_name,
                    original_content_id, original_page_number,
                    is_valid, confidence_score, generation_method
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, mapping)
    
    conn.commit()
    
    # Verify
    total = conn.execute("SELECT COUNT(*) FROM faq_content_map").fetchone()[0]
    valid = conn.execute("SELECT COUNT(*) FROM faq_content_map WHERE is_valid = 1").fetchone()[0]
    invalid = total - valid
    
    print(f"✓ Inserted {total} records into faq_content_map")
    print(f"  Valid: {valid}, Invalidated: {invalid}")
    
    # Show sample
    print("\nFAQ mappings:")
    rows = conn.execute("""
        SELECT map_id, question_id, current_page_number, is_valid, invalidation_reason
        FROM faq_content_map
        ORDER BY map_id
    """).fetchall()
    
    for row in rows:
        status = "VALID" if row[3] else f"INVALID ({row[4]})"
        print(f"  Map {row[0]}: Q{row[1]} on page {row[2]} - {status}")

## Step 11: Insert Sample Data - content_diffs

Storing diff for the content_edit change

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Create sample diff content
    diff_text = """--- old/page_2.md
+++ new/page_2.md
@@ -1,5 +1,5 @@
 # Leave Policy
 
-Employees are entitled to 15 days of annual leave.
+Employees are entitled to 20 days of annual leave.
 
 Additional benefits include:
"""
    
    # Compress the diff
    diff_compressed = gzip.compress(diff_text.encode('utf-8'))
    
    # Create semantic summary (JSON)
    semantic_summary = json.dumps({
        "summary": ["Annual leave days changed from 15 to 20"],
        "total_changes": 1,
        "stats": {
            "headers_changed": 0,
            "paragraphs_changed": 1,
            "list_items_changed": 0,
            "links_changed": 0
        }
    })
    
    # Insert diff for change_id = 4 (the content_edit)
    checksum_2 = 'b' * 64
    checksum_2_edited = 'd' * 64
    
    conn.execute("""
        INSERT INTO content_diffs (
            change_id, old_checksum, new_checksum, unified_diff_gzip,
            similarity_score, chars_added, chars_removed, lines_added, lines_removed,
            is_minor_change, semantic_summary, processing_time_ms
        ) VALUES (4, ?, ?, ?, 0.95, 2, 2, 1, 1, 1, ?, 125)
    """, (checksum_2, checksum_2_edited, diff_compressed, semantic_summary))
    
    conn.commit()
    
    # Verify
    count = conn.execute("SELECT COUNT(*) FROM content_diffs").fetchone()[0]
    print(f"✓ Inserted {count} record into content_diffs")
    
    # Show details
    row = conn.execute("""
        SELECT change_id, similarity_score, chars_added, chars_removed, is_minor_change
        FROM content_diffs
    """).fetchone()
    
    print(f"\nDiff details:")
    print(f"  Change ID: {row[0]}")
    print(f"  Similarity: {row[1]:.2%}")
    print(f"  Changes: +{row[2]} / -{row[3]} chars")
    print(f"  Minor change: {bool(row[4])}")
    print(f"  Compressed diff size: {len(diff_compressed)} bytes")

## Step 12: Test Foreign Key Constraints

Verify that FK constraints are working correctly

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    print("Testing Foreign Key Constraints...\n")
    
    # Test 1: Try to insert change_log with non-existent content_id
    print("Test 1: Insert content_change_log with invalid content_id (should FAIL)")
    try:
        conn.execute("""
            INSERT INTO content_change_log (
                content_id, file_name, page_number, content_checksum,
                change_type, effective_date
            ) VALUES (999, 'test.pdf', 1, ?, 'initial_creation', '2025-01-01T00:00:00Z')
        """, ('e' * 64,))
        print("  ✗ FAIL: Insert succeeded (FK constraint not working!)")
    except sqlite3.IntegrityError as e:
        print(f"  ✓ PASS: Insert blocked by FK constraint")
        print(f"    Error: {e}")
    
    # Test 2: Try to insert faq_content_map with non-existent question_id
    print("\nTest 2: Insert faq_content_map with invalid question_id (should FAIL)")
    try:
        conn.execute("""
            INSERT INTO faq_content_map (
                question_id, content_checksum, current_content_id,
                current_page_number, current_file_name,
                original_content_id, original_page_number
            ) VALUES (999, ?, 1, 1, 'test.pdf', 1, 1)
        """, ('e' * 64,))
        print("  ✗ FAIL: Insert succeeded (FK constraint not working!)")
    except sqlite3.IntegrityError as e:
        print(f"  ✓ PASS: Insert blocked by FK constraint")
        print(f"    Error: {e}")
    
    # Test 3: Check foreign_key_check
    print("\nTest 3: Run PRAGMA foreign_key_check (should be empty)")
    violations = conn.execute("PRAGMA foreign_key_check").fetchall()
    if violations:
        print(f"  ✗ FAIL: Found {len(violations)} FK violations:")
        for v in violations:
            print(f"    {v}")
    else:
        print("  ✓ PASS: No FK violations found")
    
    print("\n✓ All FK constraint tests completed")

## Step 13: Test Trigger

Verify content_diffs trigger only allows diffs for content_edit

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    print("Testing Trigger: trg_content_diffs_only_for_edits\n")
    
    # Test: Try to insert diff for change_id = 1 (initial_creation, not content_edit)
    print("Test: Insert diff for initial_creation change (should FAIL)")
    
    sample_diff = gzip.compress(b"test diff")
    
    try:
        conn.execute("""
            INSERT INTO content_diffs (
                change_id, old_checksum, new_checksum, unified_diff_gzip,
                similarity_score, chars_added, chars_removed, lines_added, lines_removed,
                is_minor_change
            ) VALUES (1, ?, ?, ?, 0.5, 10, 10, 1, 1, 0)
        """, ('e' * 64, 'f' * 64, sample_diff))
        print("  ✗ FAIL: Insert succeeded (trigger not working!)")
    except sqlite3.IntegrityError as e:
        print(f"  ✓ PASS: Insert blocked by trigger")
        print(f"    Error: {e}")
    
    print("\n✓ Trigger test completed")

## Step 14: Query Views

Test that all views are working correctly

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    print("Testing Views...\n")
    
    # Test v_current_valid_faqs
    print("1. v_current_valid_faqs:")
    rows = conn.execute("SELECT COUNT(*) FROM v_current_valid_faqs").fetchone()
    print(f"   {rows[0]} valid FAQs found")
    
    # Test v_faqs_needing_review
    print("\n2. v_faqs_needing_review:")
    rows = conn.execute("SELECT COUNT(*) FROM v_faqs_needing_review").fetchone()
    print(f"   {rows[0]} FAQs need review")
    
    # Test v_content_changes_with_diffs
    print("\n3. v_content_changes_with_diffs:")
    rows = conn.execute("""
        SELECT file_name, page_number, change_magnitude, similarity_score
        FROM v_content_changes_with_diffs
    """).fetchall()
    print(f"   {len(rows)} content edits with diffs")
    for row in rows:
        print(f"     {row[0]} page {row[1]}: {row[2]} change (similarity: {row[3]:.2%})")
    
    # Test v_pending_diffs
    print("\n4. v_pending_diffs:")
    rows = conn.execute("SELECT COUNT(*) FROM v_pending_diffs").fetchone()
    print(f"   {rows[0]} diffs pending generation")
    
    # Test v_diff_processing_stats
    print("\n5. v_diff_processing_stats:")
    row = conn.execute("""
        SELECT total_content_edits, diffs_generated, pending_diffs, minor_changes
        FROM v_diff_processing_stats
    """).fetchone()
    print(f"   Total edits: {row[0]}, Diffs: {row[1]}, Pending: {row[2]}, Minor: {row[3]}")
    
    print("\n✓ All views working correctly")

## Step 15: Summary Statistics

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    print("=" * 60)
    print("DATABASE SETUP COMPLETE")
    print("=" * 60)
    
    print(f"\nDatabase: {DB_PATH.absolute()}")
    print(f"Schema Version: 4.0 (Single Database Architecture)")
    
    # Table counts
    print("\nTable Record Counts:")
    tables = [
        'content_repo',
        'faq_questions',
        'faq_answers',
        'content_change_log',
        'faq_content_map',
        'content_diffs',
        'schema_version'
    ]
    
    for table in tables:
        count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"  {table:25} {count:5} records")
    
    # Database size
    db_size = os.path.getsize(DB_PATH)
    print(f"\nDatabase size: {db_size:,} bytes ({db_size/1024:.2f} KB)")
    
    # Foreign keys status
    fk_enabled = conn.execute("PRAGMA foreign_keys").fetchone()[0]
    print(f"Foreign keys enabled: {bool(fk_enabled)} ✓")
    
    # Component counts
    tables_count = conn.execute(
        "SELECT COUNT(*) FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%'"
    ).fetchone()[0]
    views_count = conn.execute(
        "SELECT COUNT(*) FROM sqlite_master WHERE type='view'"
    ).fetchone()[0]
    triggers_count = conn.execute(
        "SELECT COUNT(*) FROM sqlite_master WHERE type='trigger'"
    ).fetchone()[0]
    indexes_count = conn.execute(
        "SELECT COUNT(*) FROM sqlite_master WHERE type='index' AND name NOT LIKE 'sqlite_%'"
    ).fetchone()[0]
    
    print(f"\nSchema Components:")
    print(f"  Tables:   {tables_count}")
    print(f"  Views:    {views_count}")
    print(f"  Triggers: {triggers_count}")
    print(f"  Indexes:  {indexes_count}")
    
    print("\n" + "=" * 60)
    print("✓ Setup successful - Ready for use!")
    print("=" * 60)